## 控制迷宫寻宝机器人

在这个项目中，你将使用刚刚学到的知识，尝试根据要求，编写代码，来控制一个机器人，在模拟环境中行走，并找到目标宝藏。

机器人所在的模拟环境中，会包含这样几个因素：机器人的起点、障碍物、宝藏箱。你的任务包括：

1. 分析模拟环境的数据
2. 控制机器人随机行动
3. （可选）控制机器人走到终点


* 一个良好的含有注释的代码，可以让你的程序可读性更高，尝试为你自己的代码添加相应的注释。

---

---

## 第一节 分析模拟环境的数据

首先，只有足够了解机器人所在的环境，我们的机器人才能成功找到目标宝藏，因此首先我们来对机器人所在环境的数据进行分析。在这个部分，会考察你对数据结构、控制流的了解。


### 1.1 理解模拟环境数据的储存格式

首先我们思考这样的问题：如何存储模拟环境的数据呢？

我们将我们的模拟环境抽象成一个格子世界，每个格子按照坐标编号进行标记；每个格子中会有四个情况，分别为普通格子（可通行）、机器人的起点（可通行）、障碍物（不可通行）、宝藏箱（目标点）。例如，一个模拟环境就可以抽象成3行4列的格子世界，并按这按这样的方法进行存储：
```
environment = [[0,0,0,2], 
               [1,2,0,0],
               [0,2,3,2]]
```
我们用了一个列表来保存虚拟世界的数据。外层列表中的每一个元素依然是一个列表，它代表模拟环境中每行的数据。而对于这个列表中的每个元素都是一个数，它们的含义是：
- 0: 普通格子（可通行）
- 1: 机器人的起点（可通行）
- 2: 障碍物（不可通行）
- 3: 宝藏箱（目标点）

那么，根据上述的数据，这个迷宫的第二行第一列，是我们机器人的起点。

__注：我们描述的迷宫的坐标位置（第一行第一列），和迷宫下标索引的值（如 `(0,0)`）是不一样的，请注意下标的问题。__


如下的代码，使用了辅助函数，读取了模拟环境的数据，并保存在了 `env_data` 变量中。


In [2]:
import helper

env_data = helper.fetch_maze()

maze-id 1-1545545512
[[3, 2, 2, 2, 2, 2, 2, 2, 1],
 [0, 0, 2, 2, 2, 2, 2, 0, 0],
 [2, 0, 0, 2, 2, 2, 0, 0, 2],
 [2, 2, 0, 0, 2, 0, 0, 2, 2],
 [2, 2, 2, 0, 0, 0, 2, 2, 2]]


---


**任务1：**在如下代码中，请写代码获得这些值：

1. 模拟环境的长和宽
2. 模拟环境中第3行第6列元素

In [5]:
#TODO 1模拟环境的行数
rows = len(env_data) 

#TODO 2模拟环境的列数
columns = len(env_data[0]) 

#TODO 3取出模拟环境第三行第六列的元素
row_3_col_6 = env_data[2][5] 

print("迷宫共有", rows, "行", columns, "列，第三行第六列的元素是", row_3_col_6)

迷宫共有 5 行 9 列，第三行第六列的元素是 2


---

## 1.2 分析模拟环境数据

接着我们需要对模拟环境的中的数据进行分析。请根据如下的指示，计算相应的值。

---

**任务2：**在如下代码中，请计算模拟环境中，第一行和第三列的障碍物个数。

提示：*可以用循环完成。*

In [8]:
#TODO 4计算模拟环境中，第一行的的障碍物个数。
number_of_barriers_row1 = env_data[0].count(2)

#TODO 5计算模拟环境中，第三列的的障碍物个数。
number_of_barriers_col3 = list(zip(*env_data))[2].count(2)  #这个地方利用zip(*env_data)+list实现对env_data转置，再对原来的第三列统计等于2的个数
print("迷宫中，第一行共有", number_of_barriers_row1, "个障碍物，第三列共有", number_of_barriers_col3, "个障碍物。")

迷宫中，第一行共有 7 个障碍物，第三列共有 3 个障碍物。


In [9]:
%run -i -e test.py RobotControllortTestCase.test_cal_barriers

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


---

**任务3：**在如下代码中：

1. 创建一个名为 `loc_map` 的字典，它有两个键值，分别为 `start` 和 `destination`，对应的值分别为起点和目标点的坐标，它们以如 `(0,0)` 的形式保存为元组。
2. 从字典中取出 `start` 对应的值，保存在 `robot_current_loc` 对应的变量中，这个变量表示小车现在的位置。

In [42]:
loc_map = {} #TODO 6按照上述要求创建字典
for row in range(len(env_data)):
    for column in range(len(env_data[0])):
        if env_data[row][column] == 1:
            loc_map['start']=(row,column)
        elif env_data[row][column] == 3:
            loc_map['destination'] = (row,column)

robot_current_loc = loc_map['start'] #TODO 7保存机器人当前的位置

In [43]:
%run -i -e test.py RobotControllortTestCase.test_cal_loc_map

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK



---

---

## 第二节 控制机器人随机漫步

在这一步中，你需发出指令，控制机器人在环境中随机行动。它会考察你对控制流、调用函数的知识。



## 2.1 控制机器人行动

我们的机器人能够执行四个动作：向上走 `u`、向下走 `d`、向左走 `l`、向右走 `r`。但是，由于有障碍，很多时候机器人的行动并不能成功。所以在这里，你需要实现一个函数，来判断机器人在某个位置，执行某个移动动作是否可行。

---

**任务4：**在下方代码中，实现名为 `is_move_valid_special` 的函数，它有两个输入，分别为机器人所在的位置坐标 `loc`，以及即将执行的动作 `act`，如 `(1,1)` 及 `u`。接着它的返回是一个布尔值，表明小车在 `loc` 位置下，是否可以执行动作 `act`。


提示1：*可以读取上方定义的 `env_data` 变量，来读取模拟环境的数据。*

提示2：*在实现函数后，请删去下方的 `pass` 代码。*

提示3：*我们需要处理边界的情况，即机器人走到了虚拟环境边界时，是不能够走出虚拟环境的。*

In [10]:
def is_move_valid_special(loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 8
    x, y = loc
    if act == 'u':
        x -= 1
    elif act == 'd':
        x += 1
    elif act == 'l':
        y -= 1
    elif act == 'r':
        y += 1
    return (0 <= y <= columns - 1) and (0 <= x <= rows - 1) and (env_data[x][y] != 2)

In [11]:
%run -i -e test.py RobotControllortTestCase.test_is_move_valid_special

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


---
**任务5：**在下方代码中，重新实现一个名为 `is_move_valid` 的函数，它有三个输入，分别为模拟环境的数据 `env_data`、机器人所在的位置坐标 `loc`、以及即将执行的动作 `act`。它的返回值与此前一样，是一个布尔值，表明小车在给定的虚拟环境中的 `loc` 位置下，是否可以执行动作 `act`。

In [12]:
def is_move_valid(env_data, loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    env -- list, the environment data
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 9
    x, y = loc
    if act == 'u':
        x -= 1
    elif act == 'd':
        x += 1
    elif act == 'l':
        y -= 1
    elif act == 'r':
        y += 1
    return (0 <= y <= columns - 1) and (0 <= x <= rows - 1) and (env_data[x][y] != 2)

In [13]:
%run -i -e test.py RobotControllortTestCase.test_is_move_valid

.
----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


---

**任务6：**请回答：
  1. 在任务4及任务5中的实现的两个函数中，`env_data` 这个变量有什么不同？
  2. 调用``is_move_valid``函数，参数为``env_data_``、``loc_``、``act_``，如果在函数内修改``env_data``是否会改变``env_data_``的值？为什么？
  
提示：_可以尝试从变量作用域的角度回答该问题1。_


提示：_可以尝试从可变类型变量和不可变类型变量的角度回答该问题2。_



**回答：** 1.任务4中调用的env_data是全局变量，作用域是全局范围，任务5中调用的env_data是is_move_valid函数的局部变量，只可以在本函数中调用，如果将任务5中函数参数env_data改为其他的与env_data的名称，你会发现在is_move_valid函数外部的任务地方访问它都会报错；
2.在函数内修改env_data会改变env_data的值，首先env_data的类型为列表，列表为可变类型变量，我们可以修改其中的任意值。其次，在is_move_valid函数内部，我们对env_data进行修改，修改的结果会直接传递给它的实参，实现对全局变量env_data的修改，另外，你会发现两者通过id()打印出来的地址是相同的。

---

## 2.2 机器人可行动作

---

**任务7：**编写一个名为 `valid_actions` 的函数。它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`，输出是一个列表，表明机器人在这个位置所有的可行动作。

提示：*可以尝试调用上方定义的`is_move_valid`函数。*


In [19]:
## TODO 10 从头定义、实现你的函数
def valid_actions(env_data,loc):
    valid_act = []
    for act in ['u','d','l','r']:
        if is_move_valid(env_data,loc,act):
            valid_act.append(act)
    return valid_act

In [20]:
%run -i -e test.py RobotControllortTestCase.test_valid_actions

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


---

## 2.3 移动机器人

当机器人收到一个动作的时候，你机器人的位置应发生相应的变化。

**任务8：**编写一个名为 `move_robot` 的函数，它有两个输入，分别为机器人当前所在的位置 `loc` 和即将执行的动作 `act`。接着会返回机器人执行动作之后的新位置 `new_loc`。

In [21]:
##TODO 11 从头定义、实现你的函数
def move_robot(loc,act):
    move_dict = {
        'u': (-1,0),                     #利用字典定义每一种动作将引起的坐标改变，再将act作为keyname直接从move_dice中调用相应的值与loc的x，y相加
        'd': (1,0),                      #这个点子很赞哦！
        'l': (0,-1),
        'r': (0,1)
    }

    return loc[0] + move_dict[act][0], loc[1] + move_dict[act][1]      

In [22]:
%run -i -e test.py RobotControllortTestCase.test_move_robot

.
----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


---

## 2.4 随机移动机器人

接着，我们尝试在虚拟环境中随机移动机器人，看看会有什么效果。

**任务9：**编写一个名为 `random_choose_actions` 的函数，它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`。机器人会执行一个300次的循环，每次循环，他会执行以下任务：

1. 利用上方定义的 `valid_actions` 函数，找出当前位置下，机器人可行的动作；
2. 利用 `random` 库中的 `choice` 函数，从机器人可行的动作中，随机挑选出一个动作；
3. 接着根据这个动作，利用上方定义的 `move_robot` 函数，来移动机器人，并更新机器人的位置；
4. 当机器人走到终点时，输出“在第n个回合找到宝藏！”。

提示：如果机器人无法在300个回合内找到宝藏的话，试试看增大这个数字，也许会有不错的效果 :P

In [90]:
import random

In [91]:
##TODO 12 从头实现你的函数
def random_choose_actions(env_data,loc):
    score = 0
    for i in range(300):
        if env_data[loc[0]][loc[1]] == 3:
            break
        else:
            valid_action = valid_actions(env_data,loc)
            action = random.choice(valid_action)
            loc = move_robot(loc,action)
    print('在第{}个回合找到宝藏'.format(i-1))

In [92]:
# 运行
random_choose_actions(env_data, robot_current_loc)

(0, 8) u
(0, 8) l
(1, 8) d
(1, 7) u
(1, 7) l
(2, 7) d
(2, 7) r
(2, 6) u
(2, 6) l
(3, 6) d
(3, 6) r
(3, 5) u
(3, 5) l
(3, 6) d
(3, 6) r
(3, 5) u
(3, 5) l
(3, 6) d
(3, 6) r
(3, 5) u
(3, 5) l
(4, 5) r
(4, 4) u
(4, 5) r
(4, 4) u
(4, 3) l
(4, 4) u
(4, 3) l
(4, 4) u
(4, 3) l
(4, 4) u
(4, 5) r
(4, 4) u
(4, 5) r
(3, 5) u
(3, 5) l
(4, 5) r
(4, 4) u
(4, 5) r
(3, 5) u
(3, 5) l
(3, 6) d
(3, 6) r
(2, 6) u
(2, 6) l
(3, 6) d
(3, 6) r
(2, 6) u
(2, 6) l
(2, 7) d
(2, 7) r
(1, 7) u
(1, 7) l
(2, 7) d
(2, 7) r
(1, 7) u
(1, 7) l
(1, 8) d
(1, 7) u
(1, 7) l
(2, 7) d
(2, 7) r
(1, 7) u
(1, 7) l
(2, 7) d
(2, 7) r
(2, 6) u
(2, 6) l
(3, 6) d
(3, 6) r
(3, 5) u
(3, 5) l
(4, 5) r
(3, 5) u
(3, 5) l
(3, 6) d
(3, 6) r
(2, 6) u
(2, 6) l
(2, 7) d
(2, 7) r
(1, 7) u
(1, 7) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(0, 8) u
(0, 8) l
(1, 8) d
(1, 7) u
(1, 7) l
(1, 8) d
(1, 7) u
(1, 7) l
(2, 7) d
(2, 7) r
(1, 7) u
(1, 7) l
(


---

---

## （可选）第三节 控制机器人走到终点

## 3.1 控制机器人走到终点

在这里，你将综合上述的知识，编码控制机器人走到终点。这个任务对刚刚入门的你来说可能有些挑战，所以它是一个选做题。

**任务10**：尝试实现一个算法，能够对给定的模拟环境，输出机器人的行动策略，使之能够走到终点。

提示：_你可以尝试参考：_
* 深度/广度优先算法。
    以及以下参考资料：
    1. https://blog.csdn.net/raphealguo/article/details/7523411 
    2. https://www.cnblogs.com/yupeng/p/3414736.html 
* A星算法。
    以及以下参考资料：
    1. https://baike.baidu.com/item/A%2A算法 
    2. https://blog.csdn.net/hitwhylz/article/details/23089415

In [26]:
##TODO 13 实现你的算法
def compute_g_value(loc_destination,valid_action_loc):
    g_value=[]
    #本项目因为不能斜向移动，u,d,l,r四个方向的移动代价都是一样的，因此只考虑从指定点到终点的估算成本
    for loc in valid_action_loc:
        g_value.append((abs(loc_destination[0]-loc[0])*10,abs(loc_destination[1]-loc[1])*10))     
    return g_value

def pre_next_move(valid_action_loc,g_value,close_list):
    next_move=[]
    new_g_value=[]
    for i in range(len(g_value)):
        if valid_action_loc[i] not in close_list:
            next_move.append(valid_action_loc[i])
            new_g_value.append(g_value[i])
    min_index = new_g_value.index(min(new_g_value))
    next_loc = next_move[min_index]
    return next_loc


close_list=[]   
loc,loc_destination = get_start_destination(env_data)
close_list.append(loc)
step = 0
#直到当前坐标等于目标坐标时，说明已移动到目标位置，退出循环，并打印当前坐标和一共移动的次数
while loc != loc_destination:
    #步数累加器
    step += 1
    #根据当前位置和地图，获得不碰触障碍物和边界的行动方案，返回包含u,d,l,r（不一定是全部）的List
    valid_action = valid_actions(env_data,loc)
    #根据当前位置和可能的行动方案，获得每一个行动方案下一步的坐标，得到一个List
    for action in valid_action:
        valid_action_loc.append(move_robot(loc,action))
    #计算下一步每个行动方案距离目标坐标的G值，得到一个list
    g_value = compute_g_value(loc_destination,valid_action_loc)
    #将不在close_list中的location放入新的可执行移动列表中，并对应生成一个g_value的list,找到最小的g_value，并返回该g_value对应的next_loc
    loc = pre_next_move(valid_action_loc,g_value,close_list)
    print(loc)
    #此时已用下一个位置，更新了当前位置，我们同时将当前位置关闭，添加到close_list中
    close_list.append(loc)
print(loc,step)

(1, 8)
(1, 7)
(2, 7)
(2, 6)
(3, 6)
(3, 5)
(4, 5)
(4, 4)
(4, 3)
(3, 3)
(3, 2)
(2, 2)
(2, 1)
(1, 1)
(1, 0)
(0, 0)
(0, 0) 16


In [17]:
def get_start_destination(env_data):
    loc_map = {}
    for row in range(len(env_data)):
        for column in range(len(env_data[0])):
            if env_data[row][column] == 1:
                loc_map['start']=(row,column)
            elif env_data[row][column] == 3:
                loc_map['destination'] = (row,column)
    return loc_map['start'],loc_map['destination']

> 注意: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。